In [1]:
import glob
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
import numpy as np
from itertools import chain 
from IPython.display import clear_output
from ipywidgets import widgets
from IPython.display import display
from sklearn.model_selection import train_test_split
import os
from sklearn.neural_network import MLPClassifier
import numpy as np
import scipy
%autosave 60

Autosaving every 60 seconds


In [5]:
output = None

In [8]:
def progress(count,total):
        global output
        
        #display(widgets.IntSlider())
        #text = widgets.Text()

        with output:
            print "{0:.2f}".format(count*100.0/total)+" %"
            print '|'+'='*int(count*100.0/total)+'>'+'.'*(100-int(count*100.0/total))+'|'
        with output:
            clear_output(wait=True)

In [41]:
path = '/home/rakesh/Desktop/zCap/public_dataset/HMOG/990622/'

filter_file = 'TouchEvent.csv'
out_path = '/home/rakesh/OUTPUT/TouchEvent/'

In [42]:
def concat(path,filter_file):
    df = None
    l = []
    
    for session in glob.glob(path+'*'):
    
        for f in glob.glob(session+'/*'):
        
            if f.split('/')[-1]==filter_file:
                df = pd.read_csv(f,header=None)
                l.append(df)
    return pd.concat(l).sort_values(by=[0]) ## sort by absolute timestamp

In [43]:
mdf = concat(path,filter_file)

In [44]:
mdf = mdf.drop(columns=[1,2])

In [45]:
start = mdf.iloc[0][0]

gap = 10       # gap of 10ms
count = 0
touch_list = []
tmp = []
length = len(mdf)

output = widgets.Output()
display(output)

for index in range(0,length):
    
    if index%2000==0:
        progress(index,length)
        #print "{:.2f} and {}".format((index*1.0)/length,index),"|",
    
    
    row = mdf.iloc[index].tolist()
    count+=1
    cur_time = row[0]
    if cur_time<start:
        print "Yes"
    
    if cur_time>=start and cur_time<= start+gap:
        
        tmp.append(row[1:])

    else:
        
        start = cur_time
        count+=1
        if len(tmp)!=0:
            
            df = pd.DataFrame(tmp)
            
            x0,x1 = df[0].mode()[0], df[1].mode()[0]    ## pointer count, pointerid
            x2,x3,x4 = df[3].mean(),df[3].max(),df[3].min()      # x-axis
            x5,x6,x7 = df[4].mean(), df[4].max(), df[4].min()  # y-axis
            x8,x9 = df[5].mean(),df[5].max()                ## pressure
            x10,x11 = df[6].mean(),df[6].max()                ##contact size   
            x12 = df[7].mode()[0]                         # phone orientation
            #x13 = len(df)
            tuple_list = [x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12]
            try:
                tuple_list = [float(i) for i in tuple_list]
            except:
                print "\n\n",tuple_list
                break
            #print tuple_list
            touch_list.append(tuple_list)
        tmp = []
        
        tmp.append(row[1:])
print count

Output()

248666


In [46]:
touch = pd.DataFrame(touch_list)
print touch.shape
touch = touch.drop_duplicates()

(101054, 13)


In [47]:
touch.to_csv(out_path+'990622_touchevent.csv',sep=',',header = False)